In [26]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import random
import google.generativeai
import re
import json
import time

In [27]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCl
DeepSeek API Key exists and begins sk-


In [40]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

google.generativeai.configure()

# --- Modelos ---
gpt_model = "gpt-4.1-mini"
claude_model = "claude-sonnet-4-20250514" #claude-3-5-haiku-latest
deepseek_model = "deepseek-chat"

In [29]:
# ---------- CONFIGURAÇÃO (edite se quiser) ----------
MIN_VAL = 1
MAX_VAL = 100          # altere para 200 ou 500 para testar escalabilidade
N_ROUNDS = 7
MAX_ATTEMPTS = 5

In [30]:
# ---------- HELPERS ----------
def extract_integer_from_text(text, min_v=MIN_VAL, max_v=MAX_VAL):
    """Tenta extrair um inteiro válido do texto (mais tolerante)."""
    if not text:
        return None
    # procura números (com sinal)
    nums = re.findall(r'-?\d+', text)
    for n in nums:
        try:
            v = int(n)
            if min_v <= v <= max_v:
                return v
        except:
            continue
    # se não achar, tenta mapear palavras (opcional) - omitido por simplicidade
    return None

def pretty_print_round(player_name, round_i, attempt_i, guess, feedback, secret=None):
    fb = feedback
    s = f"[{player_name}] Rodada {round_i}/{N_ROUNDS} — Tentativa {attempt_i}/{MAX_ATTEMPTS}: chute={guess} -> {fb}"
    if secret is not None:
        s += f" (secreto={secret})"
    print(s)

def summarize_results(results):
    wins = sum(1 for r in results if r['acertou'])
    return {"wins": wins, "rounds": len(results)}

In [31]:
# ---------- Função para pedir chute ao GPT (stream se disponível) ----------
def ask_gpt_for_guess(previous_attempts, previous_feedbacks, min_v=MIN_VAL, max_v=MAX_VAL):
    """
    Envia ao GPT o estado atual e solicita UM ÚNICO número entre min_v e max_v.
    Retorna (guess:int or None, raw_text:str)
    """
    # system prompt — instrui a saída estrita
    system_prompt = f"""
Você é um jogador humano simulando um jogador real. Seu objetivo: escolher o próximo palpite (APENAS UM NÚMERO inteiro) entre {min_v} e {max_v} para o jogo do marciano.
Você receberá o histórico de tentativas e dicas.
**Responda estritamente** com um único número inteiro dentro do intervalo. Opcionalmente você pode adicionar uma linha curta começando com "RAZÃO:" para justificar — mas a primeira linha deve conter **apenas** o número.
Nada além disso.
"""
    user_msg = f"Histórico:\nTentativas: {previous_attempts}\nDicas: {previous_feedbacks}\nDê o PRÓXIMO palpite (um único número inteiro)."
    # request streaming; se o seu cliente openai não suportar stream=True, remova a opção
    try:
        resp_stream = openai.chat.completions.create(
            model=gpt_model,
            messages=[{"role":"system","content":system_prompt}, {"role":"user","content":user_msg}],
            stream=True
        )
        text = ""
        for chunk in resp_stream:
            delta = chunk.choices[0].delta
            if getattr(delta, "content", None):
                print(delta.content, end="", flush=True)
                text += delta.content
        print()  # nova linha
    except Exception as e:
        # fallback para chamada não-streaming
        resp = openai.chat.completions.create(
            model=gpt_model,
            messages=[{"role":"system","content":system_prompt}, {"role":"user","content":user_msg}]
        )
        text = resp.choices[0].message["content"]
        print(text)
    guess = extract_integer_from_text(text, min_v, max_v)
    return guess, text

In [32]:
# ---------- Função que executa N_ROUNDS para um "modelo que devolve palpites" (ex: GPT) ----------
def play_model_by_asking_gpt(player_name="ChatGPT"):
    resultados = []
    for r in range(1, N_ROUNDS+1):
        secret = random.randint(MIN_VAL, MAX_VAL)
        tentativas = []
        dicas = []
        acertou = False

        for attempt in range(1, MAX_ATTEMPTS+1):
            guess, raw = ask_gpt_for_guess(tentativas, dicas, MIN_VAL, MAX_VAL)
            # se não extraiu número, usamos fallback (tentar novamente ou chute aleatório)
            if guess is None:
                print(f"[{player_name}] resposta não continha número válido, usando fallback aleatório.")
                guess = random.randint(MIN_VAL, MAX_VAL)
            # registra
            if guess in tentativas:
                # evitar repetir exatamente o mesmo número (é permitido, mas avisamos)
                pass
            tentativas.append(guess)
            # avalia
            if guess == secret:
                feedback = "acertou"
                dicas.append(feedback)
                pretty_print_round(player_name, r, attempt, guess, feedback)
                acertou = True
                break
            elif guess > secret:
                feedback = "mais à esquerda"   # precisa chutar menor
                dicas.append(feedback)
                pretty_print_round(player_name, r, attempt, guess, feedback)
            else:
                feedback = "mais à direita"   # precisa chutar maior
                dicas.append(feedback)
                pretty_print_round(player_name, r, attempt, guess, feedback)

        resultados.append({
            "rodada": r,
            "secreto": secret,
            "tentativas": tentativas,
            "dicas": dicas,
            "acertou": acertou
        })
        # pequeno delay para legibilidade do stream
        time.sleep(0.12)
    return resultados


In [33]:
# ---------- DEEPSEEK: pedir estratégia (JSON) com base nos resultados do GPT ----------
def request_deepseek_strategy(gpt_results, min_v=MIN_VAL, max_v=MAX_VAL):
    """
    Solicita ao DeepSeek uma estratégia humana e estruturada (JSON).
    Esperamos um JSON com: name, type, description, params, human_steps, pseudocode
    """
    system_prompt = f"""
Você é o DeepSeek. Sua tarefa:
1) Analise os resultados do ChatGPT (dados JSON) e proponha UMA estratégia que possa ser aplicada por um jogador humano para aumentar a taxa de acerto no jogo do marciano.
2) A estratégia deve ser escalável para ranges maiores (por exemplo 1-{min_v} até 1-{max_v}, e se for aplicável, diga como adaptar para 1-200/1-500).
3) **Retorne estritamente apenas um JSON válido** com as chaves:
   - name (string)
   - type ( "binary", "interval", "fixed_sequence", "probabilistic" ou "other" )
   - description (breve)
   - params (objeto de parâmetros se houver)
   - human_steps (lista de passos claros e simples para um humano seguir)
   - pseudocode (string curta com pseudo-código)
Se não puder pensar em algo melhor que busca binária, retorne a busca binária estruturada.
"""
    user_msg = "Resultados do ChatGPT (JSON):\n" + json.dumps(gpt_results, indent=2) + f"\nFaixa: {min_v}-{max_v}\nResponda com JSON."

    # pedimos sem stream para facilitar parser
    try:
        resp = deepseek.chat.completions.create(
            model=deepseek_model,
            messages=[{"role":"system","content":system_prompt}, {"role":"user","content":user_msg}],
            max_tokens=800
        )
        text = resp.choices[0].message.content
    except Exception as e:
        # fallback: se deepseek API não responder por algum motivo, devolvemos estratégia padrão (busca binária)
        print("Erro pedindo estratégia ao DeepSeek:", e)
        text = None

    # tenta extrair JSON do texto retornado
    strategy = None
    if text:
        # extrai o primeiro bloco JSON do texto
        m = re.search(r'(\{.*\})', text, flags=re.DOTALL)
        if m:
            candidate = m.group(1)
            try:
                strategy = json.loads(candidate)
            except Exception as e:
                print("Falha ao parsear JSON retornado pelo DeepSeek — fará fallback para busca binária.")
                strategy = None
        else:
            print("DeepSeek não retornou JSON detectável — fallback para busca binária.")
            strategy = None

    # fallback strategy (binary)
    if strategy is None:
        strategy = {
            "name": "Busca Binária (fallback)",
            "type": "binary",
            "description": "Busca binária clássica: comece pelo meio, reduza intervalo conforme dicas.",
            "params": {},
            "human_steps": [
                "Defina low=min, high=max.",
                "Chute o ponto médio (floor((low+high)/2)).",
                "Recebendo dica 'mais à esquerda' reduza high = guess-1.",
                "Recebendo 'mais à direita' aumente low = guess+1.",
                "Repita até esgotar tentativas."
            ],
            "pseudocode": "while attempts: guess=(low+high)//2; if guess>secret: high=guess-1 elif guess<secret: low=guess+1"
        }

    # Mostra ao usuário a estratégia recebida (raw)
    print("\n--- Estratégia DeepSeek (raw) ---")
    print(json.dumps(strategy, indent=2, ensure_ascii=False))
    print("--- Fim da estratégia ---\n")
    return strategy


In [34]:
# ---------- Claude: pedir estratégia (JSON) com base nos resultados do GPT ----------
def request_claude_strategy(claude_text, min_v=MIN_VAL, max_v=MAX_VAL):
    """
    Solicita ao Claude uma estratégia humana e estruturada (JSON).
    Esperamos um JSON com: name, type, description, params, human_steps, pseudocode
    """
    system_prompt = f"""
Você é o Claude. Sua tarefa:
1) Analise o relatório feito das jogadas anteriores e proponha UMA estratégia que possa ser aplicada por um jogador humano para aumentar a taxa de acerto no jogo do marciano.
2) A estratégia deve ser escalável para ranges maiores (por exemplo 1-{min_v} até 1-{max_v}, e se for aplicável, diga como adaptar para 1-200/1-500).
3) **Retorne estritamente apenas um JSON válido** com as chaves:
   - name (string)
   - type ( "binary", "interval", "fixed_sequence", "probabilistic" ou "other" )
   - description (breve)
   - params (objeto de parâmetros se houver)
   - human_steps (lista de passos claros e simples para um humano seguir)
   - pseudocode (string curta com pseudo-código)
Se não puder pensar em algo melhor que busca binária, retorne a busca binária estruturada.
"""
    user_msg = "Relatório feito:\n" + json.dumps(claude_text, indent=2) + f"\nFaixa: {min_v}-{max_v}\nResponda com JSON."

    # pedimos sem stream para facilitar parser
    try:
        ans = claude.messages.create(
            model=claude_model,
            system=system_prompt,
            messages=[{"role":"user","content":user_msg}],
            max_tokens=1000
        )
        # dependendo da API, acessar texto pode variar
        text = ans.content[0].text if hasattr(ans, "content") else ans.choices[0].message.content
    except Exception as e:
        # fallback: se deepseek API não responder por algum motivo, devolvemos estratégia padrão (busca binária)
        print("Erro pedindo estratégia ao Claude:", e)
        text = None

    # tenta extrair JSON do texto retornado
    strategy = None
    if text:
        # extrai o primeiro bloco JSON do texto
        m = re.search(r'(\{.*\})', text, flags=re.DOTALL)
        if m:
            candidate = m.group(1)
            try:
                strategy = json.loads(candidate)
            except Exception as e:
                print("Falha ao parsear JSON retornado pelo DeepSeek — fará fallback para busca binária.")
                strategy = None
        else:
            print("DeepSeek não retornou JSON detectável — fallback para busca binária.")
            strategy = None

    # fallback strategy (binary)
    if strategy is None:
        strategy = {
            "name": "Busca Binária (fallback)",
            "type": "binary",
            "description": "Busca binária clássica: comece pelo meio, reduza intervalo conforme dicas.",
            "params": {},
            "human_steps": [
                "Defina low=min, high=max.",
                "Chute o ponto médio (floor((low+high)/2)).",
                "Recebendo dica 'mais à esquerda' reduza high = guess-1.",
                "Recebendo 'mais à direita' aumente low = guess+1.",
                "Repita até esgotar tentativas."
            ],
            "pseudocode": "while attempts: guess=(low+high)//2; if guess>secret: high=guess-1 elif guess<secret: low=guess+1"
        }

    # Mostra ao usuário a estratégia recebida (raw)
    print("\n--- Estratégia DeepSeek (raw) ---")
    print(json.dumps(strategy, indent=2, ensure_ascii=False))
    print("--- Fim da estratégia ---\n")
    return strategy


In [35]:
# ---------- Executor: converte a estratégia em função que devolve palpites ----------
def make_strategy_fn_from_spec(spec, min_v=MIN_VAL, max_v=MAX_VAL):
    """
    Retorna uma função strategy_fn(tentativas, dicas) -> next_guess
    Implementa 'binary', 'interval', 'fixed_sequence' se possível. Caso desconhecido -> binary fallback.
    """
    t = spec.get("type", "").lower()
    if t == "binary":
        def fn(tentativas, dicas):
            low, high = min_v, max_v
            for g, fb in zip(tentativas, dicas):
                if fb == "mais à esquerda":
                    high = min(high, g-1)
                elif fb == "mais à direita":
                    low = max(low, g+1)
            if low > high:
                # intervalo inválido: retorna um aleatório no min/max
                return random.randint(min_v, max_v)
            return (low + high) // 2
        return fn

    if t == "fixed_sequence":
        seq = spec.get("params", {}).get("sequence", [])
        def fn(tentativas, dicas):
            # escolhe próximo da sequência que ainda não foi tentado
            for s in seq:
                if s not in tentativas and min_v <= s <= max_v:
                    return s
            # fallback
            return random.randint(min_v, max_v)
        return fn

    if t == "interval":
        # interval strategy: split range into k blocks and probe centers
        k = int(spec.get("params", {}).get("blocks", 4)) if spec.get("params") else 4
        def fn(tentativas, dicas):
            # recompute blocks based on feedback
            low, high = min_v, max_v
            for g, fb in zip(tentativas, dicas):
                if fb == "mais à esquerda":
                    high = min(high, g-1)
                elif fb == "mais à direita":
                    low = max(low, g+1)
            if low > high:
                return random.randint(min_v, max_v)
            block_size = max(1, (high - low + 1) // k)
            # choose center of leftmost block not yet probed
            for i in range(k):
                b_low = low + i*block_size
                b_high = min(high, b_low + block_size - 1)
                if b_low > b_high: 
                    continue
                center = (b_low + b_high) // 2
                if center not in tentativas:
                    return center
            return (low + high) // 2
        return fn

    # default fallback binary
    return make_strategy_fn_from_spec({"type":"binary"}, min_v, max_v)



In [36]:
# ---------- JOGA com a estratégia definida (usado para DeepSeek) ----------
def play_with_strategy_fn(strategy_fn, player_name="DeepSeek"):
    resultados = []
    for r in range(1, N_ROUNDS+1):
        secret = random.randint(MIN_VAL, MAX_VAL)
        tentativas = []
        dicas = []
        acertou = False
        for attempt in range(1, MAX_ATTEMPTS+1):
            guess = strategy_fn(tentativas, dicas)
            # valida range
            if guess is None or not (MIN_VAL <= guess <= MAX_VAL):
                guess = random.randint(MIN_VAL, MAX_VAL)
            tentativas.append(guess)
            # avalia
            if guess == secret:
                feedback = "acertou"
                dicas.append(feedback)
                pretty_print_round(player_name, r, attempt, guess, feedback)
                acertou = True
                break
            elif guess > secret:
                feedback = "mais à esquerda"
                dicas.append(feedback)
                pretty_print_round(player_name, r, attempt, guess, feedback)
            else:
                feedback = "mais à direita"
                dicas.append(feedback)
                pretty_print_round(player_name, r, attempt, guess, feedback)
        resultados.append({
            "rodada": r,
            "secreto": secret,
            "tentativas": tentativas,
            "dicas": dicas,
            "acertou": acertou
        })
        time.sleep(0.06)
    return resultados


In [37]:
# ---------- CHAMADA AO CLAUDE para comparar (texto final) ----------
def ask_claude_to_compare(gpt_results, deepseek_results, min_v=MIN_VAL, max_v=MAX_VAL):
    system_prompt = f"""
Você é o Claude. Sua tarefa é analisar comparativamente dois agentes que jogaram o 'jogo do marciano' com {MAX_ATTEMPTS} tentativas por rodada.
Receberá os resultados do ChatGPT e do DeepSeek em JSON. Faça:
1) Resumo estatístico: vitórias/rodadas, taxa de acerto (%) e média de tentativas usadas para vitórias.
2) Análise por rodada (destacar rodadas onde um superou o outro).
3) Avalie a estratégia do DeepSeek (se fornecida), se é humana e se escala bem para ranges 1-{min_v} até 1-{max_v} e além (1-200, 1-500).
4) Sugestões práticas para melhorar ambas as IAs (ou a estratégia humana).
Responda em formato de relatório bem estruturado (seções e bullets).
"""
    user_msg = "Resultados ChatGPT:\n" + json.dumps(gpt_results, indent=2, ensure_ascii=False) + "\n\nResultados DeepSeek:\n" + json.dumps(deepseek_results, indent=2, ensure_ascii=False)
    try:
        ans = claude.messages.create(
            model=claude_model,
            system=system_prompt,
            messages=[{"role":"user","content":user_msg}],
            max_tokens=1000
        )
        # dependendo da API, acessar texto pode variar
        text = ans.content[0].text if hasattr(ans, "content") else ans.choices[0].message["content"]
    except Exception as e:
        print("Erro chamando Claude:", e)
        text = "Erro ao chamar Claude: " + str(e)
    print("\n--- Análise do Claude ---\n")
    print(text)
    print("\n--- Fim da análise ---\n")
    return text



In [38]:
# ---------- PIPELINE COMPLETA ----------
def run_experiment():
    print("\n===== ETAPA 1: ChatGPT joga (real, pedindo palpites ao modelo) =====\n")
    gpt_results = play_model_by_asking_gpt("ChatGPT")

    print("\n===== ETAPA 2: DeepSeek analisa resultados e propõe estratégia =====\n")
    strategy_spec = request_deepseek_strategy(gpt_results, MIN_VAL, MAX_VAL)

    print("\n===== ETAPA 2b: Executor aplica a estratégia proposta (DeepSeek joga realmente) =====\n")
    strategy_fn = make_strategy_fn_from_spec(strategy_spec, MIN_VAL, MAX_VAL)
    deepseek_results = play_with_strategy_fn(strategy_fn, player_name="DeepSeek")

    print("\n===== ETAPA 3: Claude analisa e compara os resultados =====\n")
    claude_text = ask_claude_to_compare(gpt_results, deepseek_results, MIN_VAL, MAX_VAL)

    print("\n===== ETAPA 4: Claude joga com uma nova estratégia =====\n")
    strategy_spec_claude = request_claude_strategy(claude_text, MIN_VAL, MAX_VAL)
    strategy_fn_claude = make_strategy_fn_from_spec(strategy_spec_claude, MIN_VAL, MAX_VAL)
    claude_results = play_with_strategy_fn(strategy_fn_claude, player_name="Claude")

    # resumo final em Python
    print("Resumo final (Python):")
    print("ChatGPT:", summarize_results(gpt_results))
    print("DeepSeek:", summarize_results(deepseek_results))
    return {
        "gpt_results": gpt_results,
        "deepseek_results": deepseek_results,
        "deepseek_strategy": strategy_spec,
        "claude_analysis": claude_text,
        "claude_results:": claude_results
    }


In [41]:
# ---- Para executar ----
resultado_total = run_experiment()
# Após colar, descomente a linha acima para rodar.


===== ETAPA 1: ChatGPT joga (real, pedindo palpites ao modelo) =====

50
RAZÃO: Primeiro palpite no meio do intervalo para dividir as possibilidades.
[ChatGPT] Rodada 1/7 — Tentativa 1/5: chute=50 -> mais à esquerda
25
RAZÃO: Como 50 foi alto, escolho o meio do intervalo 1 a 49 para otimizar a busca.
[ChatGPT] Rodada 1/7 — Tentativa 2/5: chute=25 -> mais à direita
37
RAZÃO: A dica "mais à esquerda" após 50 indica que o número é menor que 50. Depois, "mais à direita" após 25 indica que é maior que 25, então a melhor escolha é um número entre 26 e 49; 37 é o meio desse intervalo.
[ChatGPT] Rodada 1/7 — Tentativa 3/5: chute=37 -> mais à esquerda
31
RAZÃO: As dicas indicam que o número está entre 25 e 37, mais à esquerda de 37, então 31 é uma boa média.
[ChatGPT] Rodada 1/7 — Tentativa 4/5: chute=31 -> mais à direita
34
RAZÃO: Entre 31 (mais à direita) e 37 (mais à esquerda), 34 é um palpite razoável no meio do intervalo.
[ChatGPT] Rodada 1/7 — Tentativa 5/5: chute=34 -> mais à esquerda
5